In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## LSTM in PyTorch
* All inputs must be 3D tensors
* Semantics
  * axis 1  :  sequence
  * axis 2  :  indexes of instances in minibatch
  * axis 3  :  indexes of elements of input

In [108]:
lstm = nn.LSTM(3,3) # input dim is 3 , output_dim is 3

inputs =[torch.randn(1,3) for _ in range(5)]

torch.manual_seed(1)

# initialize hidden state.
hidden = (torch.randn(1,1,3),
         torch.randn(1,1,3))


for idx, i in enumerate(inputs):
    #Step through the sequence one element at a time.
    #after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1,1,-1),hidden)
    
    print("\nIteration ", idx, "\n")
    print("input : ", i,'\n',"output : ", out,"\n","hidden : ",hidden)


Iteration  0 

input :  tensor([[-0.4941, -0.6253, -1.3601]]) 
 output :  tensor([[[ 0.0720, -0.1889, -0.0973]]], grad_fn=<CatBackward>) 
 hidden :  (tensor([[[ 0.0720, -0.1889, -0.0973]]], grad_fn=<ViewBackward>), tensor([[[ 0.3048, -0.5322, -0.2897]]], grad_fn=<ViewBackward>))

Iteration  1 

input :  tensor([[0.2181, 1.4353, 0.5442]]) 
 output :  tensor([[[ 0.0142, -0.0729,  0.1147]]], grad_fn=<CatBackward>) 
 hidden :  (tensor([[[ 0.0142, -0.0729,  0.1147]]], grad_fn=<ViewBackward>), tensor([[[ 0.0515, -0.3396,  0.2308]]], grad_fn=<ViewBackward>))

Iteration  2 

input :  tensor([[-0.8215,  0.1928,  1.0067]]) 
 output :  tensor([[[ 0.0883, -0.0312,  0.1012]]], grad_fn=<CatBackward>) 
 hidden :  (tensor([[[ 0.0883, -0.0312,  0.1012]]], grad_fn=<ViewBackward>), tensor([[[ 0.2042, -0.1547,  0.2575]]], grad_fn=<ViewBackward>))

Iteration  3 

input :  tensor([[ 0.8080, -0.6473, -3.1203]]) 
 output :  tensor([[[ 0.0313, -0.1828, -0.1055]]], grad_fn=<CatBackward>) 
 hidden :  (tensor([[

In [36]:
# alternatively , can computer entire sequence all at once
inputs1 = torch.cat(inputs).view(len(inputs),1,-1)
print(inputs1.size())
torch.manual_seed(1)
hidden = (torch.randn(1,1,3),
          torch.randn(1,1,3))

out, hidden = lstm(inputs1,hidden)
print("output \n", out)
print("hidden layer \n" , hidden)

torch.Size([5, 1, 3])
output 
 tensor([[[-0.0367, -0.2349,  0.1330]],

        [[-0.0473, -0.2768, -0.0305]],

        [[-0.0981, -0.2573, -0.1590]],

        [[-0.0009, -0.1021, -0.0347]],

        [[ 0.1054, -0.1880, -0.2969]]], grad_fn=<CatBackward>)
hidden layer 
 (tensor([[[ 0.1054, -0.1880, -0.2969]]], grad_fn=<ViewBackward>), tensor([[[ 0.1601, -0.3293, -0.7057]]], grad_fn=<ViewBackward>))


# LSTM Tagger fro Part-of-Speetch Tagging

 * 

In [97]:
# Prepare data

def prepare_sequence(seq,to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data= [
    ("the dog ate the apple".split(), ["DET","NN","V","DET","NN"]),
    ("Everybody read that book".split(), ["NN","V","DET","NN"])
]

word_to_ix ={}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word]= len(word_to_ix)
print(word_to_ix)
tag_to_ix ={"DET" : 0, "NN": 1, "V":2}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6


{'the': 0, 'dog': 1, 'ate': 2, 'apple': 3, 'Everybody': 4, 'read': 5, 'that': 6, 'book': 7}


In [98]:
print(training_data)

[(['the', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']), (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]


In [114]:
# Create the model: 

class LSTMTagger(nn.Module):
    
    def __init__(self,
                 embedding_dim,
                 hidden_dim,
                 vocab_size,
                 target_size):
        super(LSTMTagger, self).__init__()
        
        # embedding         
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM takes word embedding as inputs, and outputs hidden states
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Initialize hidden states

        return(torch.zeros(1,1,self.hidden_dim),
               torch.zeros(1,1,self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence),1,-1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence),-1))
        tag_score = F.log_softmax(tag_space, dim = 1)
        return tag_score
        

In [115]:
# Train the model

model = LSTMTagger(EMBEDDING_DIM, 
                   HIDDEN_DIM,
                   len(word_to_ix),
                   len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=0.1)

# See the socre before training
# Note (i,j) is the socre for tag j for word i 

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_score = model(inputs)
    print("inputs \n")
    print(inputs)
    print("\noutputs \n")
    print(tag_score)

inputs 

tensor([0, 1, 2, 0, 3])

outputs 

tensor([[-0.8932, -1.3633, -1.0941],
        [-0.8370, -1.3715, -1.1606],
        [-0.8655, -1.4086, -1.0945],
        [-0.8908, -1.3966, -1.0722],
        [-0.8738, -1.2574, -1.2099]])


In [117]:
for epoch in range(300):
    for sentence, tags in training_data:

        # step1 - clear accumulated gradients()
        model.zero_grad()
        
        # clear the hidden state of LSTM 
        model.hidden = model.init_hidden()
        
        # step2 - input to tensor
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # step3 - forward pass
        tag_scores = model(sentence_in)
        
        # step3 - compute loss, gradients, and update paramters
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
# see the score after training
with torch.no_grad():
    for i in range(2):
        inputs = prepare_sequence(training_data[i][0], word_to_ix)
        tag_scores = model(inputs)

        print(tag_scores)
        print(training_data[i][0])
        print(training_data[i][1])

print(tag_to_ix)
        

tensor([[-0.0610, -4.6309, -3.0076],
        [-6.3039, -0.0127, -4.5331],
        [-2.7716, -3.6853, -0.0917],
        [-0.0355, -6.2252, -3.4136],
        [-6.0947, -0.0104, -4.8199]])
['the', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
tensor([[-6.4407, -0.0206, -3.9740],
        [-3.2040, -3.2308, -0.0835],
        [-0.0348, -5.9886, -3.4520],
        [-6.5204, -0.0183, -4.0972]])
['Everybody', 'read', 'that', 'book']
['NN', 'V', 'DET', 'NN']
{'DET': 0, 'NN': 1, 'V': 2}
